# SLC4A1: Spherocytosis, type 4  Jarolim, 1999

Data in this notebook derived from [Jarolim P, et al (1994) Duplication of 10 nucleotides in the erythroid band 3 (AE1) gene in a kindred with hereditary spherocytosis and band 3 protein deficiency (band 3PRAGUE). J Clin Invest. 1994 Jan;93(1):121-30. PMID:8282779](https://pubmed.ncbi.nlm.nih.gov/8282779/).

The variant is NM_000342.3(SLC4A1):c.2464_2465insCACCCAGATG (p.Val822Alafs) (ClinVar accession VCV000017758.3)
VariantValidator corrects to: NM_000342.3:c.2455_2464dupCACCCAGATG

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable, QcVisualizer
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.34


In [2]:
PMID="PMID:8282779"
title = "Duplication of 10 nucleotides in the erythroid band 3 (AE1) gene in a kindred with hereditary spherocytosis and band 3 protein deficiency (band 3PRAGUE)"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


## Clinical description

Table I shows hematological parameters in the affected family members. Before splenectomy, the affected subjects had a compensated hemolytic disease with reticulocytosis, hyperbilirubinemia, and increased osmotic fragility.

In [3]:
df = pd.read_excel("input/jarolim1994.xlsx")
df.head()

,Patient,dob,Sex,Splenectomy,Hemoglobin,Hemoglobin after splenectomy,Hematocrit,Hematocrit after splenectomy,MCV,reticulocytes,reticulocytes after splenectomy,Bilirubin (total),Bilirubin (total) after splenectomy,Osmotic fragility,NM_000342.3
0,I/1,1934,FEMALE,1975,NaN,16.3,NaN,46.1,84,NaN,2.7,NaN,12.1,0.58-0.34,c.2464_2465insCACCCAGATG
1,II/1,1959,FEMALE,1986,NaN,15.5,NaN,42.9,86,NaN,1.4,23.0,16.0,0.56-0.47,c.2464_2465insCACCCAGATG
2,II/2,1961,FEMALE,1989,15.1,16.0,43.0,43.7,74,9.1,1.3,23.2,5.9,0.62-0.32,c.2464_2465insCACCCAGATG
3,III/I,1983,FEMALE,-,15.6,NaN,40.0,NaN,71,7.6,NaN,18.8,NaN,0.58-0.38,c.2464_2465insCACCCAGATG
4,III/2,1985,MALE,-,12.9,NaN,33.3,NaN,77,5.9,NaN,31.7,NaN,0.52-0.40,c.2464_2465insCACCCAGATG


In [4]:
def create_individual(individual_id, age, sex, mcv, splenectomy=False):
    pfeatures = []
    pfeatures.append(HpTerm(hpo_id="HP:0001923", label="Reticulocytosis", observed="True"))
    pfeatures.append(HpTerm(hpo_id="HP:0003573", label="Increased total bilirubin", observed="True"))
    pfeatures.append(HpTerm(hpo_id="HP:0005502", label="Increased red cell osmotic fragility", observed="True"))
    pfeatures.append(HpTerm(hpo_id="HP:0004444", label="Spherocytosis", observed="True"))
    pfeatures.append(HpTerm(hpo_id="HP:0001878", label="Hemolytic anemia", observed="True"))
    if splenectomy:
        pfeatures.append(HpTerm(hpo_id="HP:0001744", label="Splenomegaly", observed="True"))
    if mcv < 80:
        #A normal MCV range is roughly 80–100 fl .
        pfeatures.append(HpTerm(hpo_id="HP:0025066", label="Decreased mean corpuscular volume", observed="True")) 
    disease = Disease(disease_id="OMIM:612653", disease_label="Spherocytosis, type 4")
    i = Individual(individual_id=individual_id, age=age, sex=sex, hpo_terms=pfeatures)
    i.set_disease(disease)
    return i

In [5]:
SLC4A1_transcript = "NM_000342.3"
vvalidator = VariantValidator(genome_build="hg38", transcript=SLC4A1_transcript)
var = vvalidator.encode_hgvs("c.2455_2464dupCACCCAGATG")
var.set_heterozygous()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.3%3Ac.2455_2464dupCACCCAGATG/NM_000342.3?content-type=application%2Fjson


In [6]:
individuals = []
for i, row in df.iterrows():
    #print(row)
    individual_id = row["Patient"]
    years = 1999 - int(row["dob"])
    age = f"P{years}Y"
    if row["Splenectomy"] != "-":
        splenectomy = True
    else:
        splenectomy = False
    mcv = float(row["MCV"])
    sex = row["Sex"]
    indi = create_individual(individual_id, age, sex, mcv, splenectomy)
    indi.add_variant(var)
    individuals.append(indi)

In [7]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_html()))

In [8]:
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
I/1 (FEMALE; P65Y),"Spherocytosis, type 4 (OMIM:612653)",NM_000342.3:c.2455_2464dup (heterozygous),Reticulocytosis (HP:0001923); Increased total bilirubin (HP:0003573); Increased red cell osmotic fragility (HP:0005502); Spherocytosis (HP:0004444); Hemolytic anemia (HP:0001878); Splenomegaly (HP:0001744)
II/1 (FEMALE; P40Y),"Spherocytosis, type 4 (OMIM:612653)",NM_000342.3:c.2455_2464dup (heterozygous),Reticulocytosis (HP:0001923); Increased total bilirubin (HP:0003573); Increased red cell osmotic fragility (HP:0005502); Spherocytosis (HP:0004444); Hemolytic anemia (HP:0001878); Splenomegaly (HP:0001744)
II/2 (FEMALE; P38Y),"Spherocytosis, type 4 (OMIM:612653)",NM_000342.3:c.2455_2464dup (heterozygous),Reticulocytosis (HP:0001923); Increased total bilirubin (HP:0003573); Increased red cell osmotic fragility (HP:0005502); Spherocytosis (HP:0004444); Hemolytic anemia (HP:0001878); Splenomegaly (HP:0001744); Decreased mean corpuscular volume (HP:0025066)
III/I (FEMALE; P16Y),"Spherocytosis, type 4 (OMIM:612653)",NM_000342.3:c.2455_2464dup (heterozygous),Reticulocytosis (HP:0001923); Increased total bilirubin (HP:0003573); Increased red cell osmotic fragility (HP:0005502); Spherocytosis (HP:0004444); Hemolytic anemia (HP:0001878); Decreased mean corpuscular volume (HP:0025066)
III/2 (MALE; P14Y),"Spherocytosis, type 4 (OMIM:612653)",NM_000342.3:c.2455_2464dup (heterozygous),Reticulocytosis (HP:0001923); Increased total bilirubin (HP:0003573); Increased red cell osmotic fragility (HP:0005502); Spherocytosis (HP:0004444); Hemolytic anemia (HP:0001878); Decreased mean corpuscular volume (HP:0025066)


In [9]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 5 GA4GH phenopackets to the directory phenopackets
